# Python Decorators

By definition, a decorator is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

Put simply: **decorators wrap a function, modifying its behavior.**

In [1]:
# Blanc Decorator
import functools

def decorator(func):
    @functools.wraps(func)  # Information about source function.
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

**Technical Detail**: The @functools.wraps decorator uses the function functools.update_wrapper() to update special attributes like \_\_name__ and \_\_doc__ that are used in the introspection.

## Timing Functions

In [2]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

## Registering Plugins

In [3]:
import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)

In [4]:
randomly_greet('John')

Using 'say_hello'


'Hello John'

The main benefit of this simple plugin architecture is that you do not need to maintain a list of which plugins exist. That list is created when the plugins register themselves. This makes it trivial to add a new plugin: just define the function and decorate it with @register.

Mishas example:

In [ ]:
from django.core.exceptions import PermissionDenied
from django.http import HttpResponse
 
def check_group(group_name):
    def wrap(func):
        def view(request):
            if (request.user.is_authenticated and
                request.user.group.name == group_name):
                return func(request)
            else:
                raise PermissionDenied
        return view
    return wrap
 
 
@check_group('Superuser')
def my_view(request):
    return HttpResponse("You're Super!")

# Decorating Classes

### Buildin decorators:

In [ ]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):  # Mutable property.
        """Get value of radius"""
        return self._radius

    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")

    @property
    def area(self):  # Immutable property.
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):  # Regular method.
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535

### Decorate whole class:

In [ ]:
from dataclasses import dataclass

@dataclass
class PlayingCard:
    rank: str
    suit: str

# Decorators With Arguments

In [6]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

In [7]:
@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")

In [9]:
greet('John')

Hello John
Hello John
Hello John
Hello John


In [14]:
import functools

def repeat(_func=None, *, num_times=2):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat

    if _func is None:
        return decorator_repeat
    else:
        return decorator_repeat(_func)
        

In [15]:
@repeat
def say_whee():
    print("Whee!")

In [16]:
@repeat(num_times=3)
def greet(name):
    print(f"Hello {name}")

In [17]:
say_whee()

greet("Penny")

Whee!
Whee!
Hello Penny
Hello Penny
Hello Penny


# Stateful Decorators

In [21]:
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

@count_calls
def say_whee():
    print("Whee!")

In [22]:
say_whee()

Call 1 of 'say_whee'
Whee!


In [23]:
say_whee()

Call 2 of 'say_whee'
Whee!


In [24]:
say_whee()

Call 3 of 'say_whee'
Whee!


# Classes as Decorators

In [25]:
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def say_whee():
    print("Whee!")

In [26]:
say_whee()
say_whee()
say_whee()

Call 1 of 'say_whee'
Whee!
Call 2 of 'say_whee'
Whee!
Call 3 of 'say_whee'
Whee!


# More Real World Examples

## Slowing Down Code

In [27]:
import functools
import time

def slow_down(_func=None, *, rate=1):
    """Sleep given amount of seconds before calling the function"""
    def decorator_slow_down(func):
        @functools.wraps(func)
        def wrapper_slow_down(*args, **kwargs):
            time.sleep(rate)
            return func(*args, **kwargs)
        return wrapper_slow_down

    if _func is None:
        return decorator_slow_down
    else:
        return decorator_slow_down(_func)

## Creating Singletons

In [28]:
import functools

def singleton(cls):
    """Make a class a Singleton class (only one instance)"""
    @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if not wrapper_singleton.instance:
            wrapper_singleton.instance = cls(*args, **kwargs)
        return wrapper_singleton.instance
    wrapper_singleton.instance = None
    return wrapper_singleton

@singleton
class TheOne:
    pass

# MY EXAMLES:

In [33]:
class BasicFunctions:
    
    @staticmethod
    def confirm_deletion(item):
        """Deletion confirmation."""
        confirm = input(
            "Вы уверены что хотите удалить '{}'? Y/N: ".format(item))
        if confirm.lower() == 'y':
            confirm = True
            print("\033[91m'{}' - удален. \033[0m".format(item))
        else:
            confirm = False
            print("\nВы отменили удаление.\n")
        return confirm

    @classmethod
    def confirm_deletion_decorator(cls, item_name):
        """Decorator to confirm item deletion."""
        def confirm_deletion(func):
            def wrapper(*args, **kwargs):
                condition = func(*args, **kwargs)
                if condition:
                    deleted = cls.confirm_deletion(item_name)
                else:
                    deleted = False
                return deleted
            return wrapper
        return confirm_deletion

In [34]:
class Example(BasicFunctions):
    
    @BasicFunctions.confirm_deletion_decorator("My item")
    def delete_some_item(self):
        #  ...some condition...
        condition = False
        return condition

In [35]:
Example().delete_some_item()

False